## Actividad árboles y bosques

En esta práctica aprenderás a:
- Entrenar y afinar un Árbol de Decisión.
- Construir un bosque manual de árboles y combinar sus predicciones mediante voto mayoritario.
- Observar cómo un ensamble de modelos puede superar a un solo árbol.

#### Parte 1

1. Generar datos
    - Crea un dataset en forma de lunas con:
    - make_moons(n_samples=10000, noise=0.4)

2. Dividir los datos
    - Separa en conjunto de entrenamiento y de prueba usando train_test_split().

3. Ajustar el modelo
    - Usa búsqueda en malla (grid search) con validación cruzada (clase GridSearchCV) para encontrar buenos hiperparámetros para un DecisionTreeClassifier.
    - Pista: prueba distintos valores de max_leaf_nodes.

4. Entrenar y evaluar
    - Entrena el árbol con todo el conjunto de entrenamiento usando los hiperparámetros óptimos.
    - Evalúa en el conjunto de prueba.

Deberías obtener aproximadamente 85%–87% de precisión.

In [1]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

X, y = make_moons(n_samples=10000, noise=0.4, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

param_grid = {'max_leaf_nodes': range(2, 100)}
tree_clf = DecisionTreeClassifier(random_state=42)
grid_search = GridSearchCV(tree_clf, param_grid, cv=5)
grid_search.fit(X_train, y_train)

best_tree_clf = grid_search.best_estimator_

best_tree_clf.fit(X_train, y_train)
y_pred = best_tree_clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Mejores hiperparámetros: {grid_search.best_params_}")
print(f"Precisión en el conjunto de prueba: {accuracy:.4f}")

Mejores hiperparámetros: {'max_leaf_nodes': 23}
Precisión en el conjunto de prueba: 0.8735


#### Parte 2 Crecer un bosque



1. Generar subconjuntos
    - Crea 1,000 subconjuntos del conjunto de entrenamiento, cada uno con 100 instancias seleccionadas aleatoriamente.
    - Pista: usa ShuffleSplit de Scikit-Learn.

2. Entrenar múltiples árboles
    - Entrena un DecisionTreeClassifier en cada subconjunto, usando los mejores hiperparámetros encontrados en la Parte 1.
    - Evalúa cada árbol individual en el conjunto de prueba.
    - Como fueron entrenados en conjuntos pequeños, se espera que tengan solo ≈80% de precisión.

3. Combinar predicciones
    - Para cada instancia del conjunto de prueba, recolecta las predicciones de los 1,000 árboles.
    - Conservar únicamente la predicción más frecuente usando mode() de SciPy.
    - Esto implementa un voto mayoritario.

4. Evaluar el bosque
    - Evalúa las predicciones combinadas en el conjunto de prueba.
    - Deberías obtener una precisión ligeramente mayor que la del árbol individual (+0.5% a +1.5%).

🎉 ¡Felicidades, has implementado tu propio Random Forest desde cero!

In [3]:
from sklearn.model_selection import ShuffleSplit
from scipy.stats import mode
import numpy as np

n_trees = 1000
n_instances = 100
shuffle_split = ShuffleSplit(n_splits=n_trees, test_size=len(X_train) - n_instances, random_state=42)

trees = []
accuracy_scores = []

for train_index, test_index in shuffle_split.split(X_train):
    X_subset = X_train[train_index]
    y_subset = y_train[train_index]

    tree_clf = DecisionTreeClassifier(max_leaf_nodes=grid_search.best_params_['max_leaf_nodes'], random_state=42)
    tree_clf.fit(X_subset, y_subset)
    trees.append(tree_clf)

    y_pred_individual = tree_clf.predict(X_test)
    accuracy_scores.append(accuracy_score(y_test, y_pred_individual))

print(f"Precisión promedio de los árboles individuales: {np.mean(accuracy_scores):.4f}")

y_pred_ensemble = np.empty([len(X_test), n_trees], dtype=np.int64)

for i, tree in enumerate(trees):
    y_pred_ensemble[:, i] = tree.predict(X_test)

y_pred_forest, _ = mode(y_pred_ensemble, axis=1)

accuracy_forest = accuracy_score(y_test, y_pred_forest)
print(f"Precisión del bosque: {accuracy_forest:.4f}")

Precisión promedio de los árboles individuales: 0.7988
Precisión del bosque: 0.8735
